In [2]:
import pandas as pd 

df = pd.read_csv('Food_Inspections_20250216.csv', dtype={'License #': str, 'Zip': str})

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

In [4]:
df.head()

Inspection ID                    DBA Name                    AKA Name  \
0        1106427        BLOOMING BUD DAYCARE        BLOOMING BUD DAYCARE   
1        2608378                 Babas Halal                 Babas Halal   
2        1106406    FIRST ZABIHA MEAT BAZAAR    FIRST ZABIHA MEAT BAZAAR   
3        2609909                HAPPY MARKET                HAPPY MARKET   
4        2609927  SAT KAIVAL FOOD INC/SUBWAY  SAT KAIVAL FOOD INC/SUBWAY   

  License #       Facility Type             Risk               Address  \
0   2215789  Daycare Combo 1586    Risk 1 (High)    5715 N LINCOLN AVE   
1   2684170          Restaurant    Risk 1 (High)      7901 S DAMEN AVE   
2   2232559       Grocery Store  Risk 2 (Medium)      2907 W DEVON AVE   
3   2912802       Grocery Store  Risk 2 (Medium)  2334 S WENTWORTH AVE   
4   2728400          Restaurant    Risk 1 (High)       1916 S STATE ST   

      City State    Zip Inspection Date Inspection Type             Results  \
0  CHICAGO    IL  60659      03/07/2013         License                Pass   
1  CHICAGO    IL  60620      12/03/2024       Complaint                Fail   
2  CHICAGO    IL  60659      02/20/2013         License                Fail   
3  CHICAGO    IL  60616      01/02/2025         Canvass  Pass w/ Conditions   
4  CHICAGO    IL  60616      01/02/2025         Canvass                Pass   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
import re

# Check for NaN values in 'Violations' column
df['Violations'] = df['Violations'].fillna('')
def parse_comments(text):
    parts = [part.strip() for part in text.split('|') if part.strip()]
    rows = []

    for part in parts:
        match = re.match(r"(?P<code>\d+)\.\s+(?P<category>.+?)\s+-\s+Comments:\s*(?P<comment>.+)", part)
        if match:
            rows.append({
                'violation_code': int(match.group('code')),
                'category': match.group('category').strip(),
                'comment': match.group('comment').strip()
            })
    return rows


In [31]:

# # Apply parsing and explode into rows
# parsed_rows = df['Violations'].apply(parse_comments)

# # Flatten the list of lists into a single DataFrame
# flat_df = pd.DataFrame([item for sublist in parsed_rows for item in sublist])


# # Get unique violation code and category pairs
# unique_violations = flat_df[['violation_code', 'category', 'comment']].value_counts().reset_index()

# # Display the results
# print(f"Total unique violation codes: {len(unique_violations)}")
# unique_violations

In [33]:
# Complete the function to parse comments properly
def parse_violations(df):
    # Fill NA values with empty string to avoid errors
    df['Violations'] = df['Violations'].fillna('')
    
    # Parse each violation entry
    parsed_rows = df['Violations'].apply(parse_comments)
    
    # Create a new dataframe with the parsed data
    violations_df = pd.DataFrame({
        'Inspection ID': df['Inspection ID'].repeat(parsed_rows.apply(len)),
        # 'DBA Name': df['DBA Name'].repeat(parsed_rows.apply(len)),
        # 'Inspection Date': df['Inspection Date'].repeat(parsed_rows.apply(len)),
        # 'Inspection Type': df['Inspection Type'].repeat(parsed_rows.apply(len)),
        # 'Results': df['Results'].repeat(parsed_rows.apply(len))
    })
    
    # Extract the parsed data into separate columns
    violations_data = [item for sublist in parsed_rows for item in sublist]
    
    # Create a dataframe from the parsed violations
    violations_details = pd.DataFrame(violations_data)
    
    # Combine the inspection info with violation details
    result_df = pd.concat([violations_df.reset_index(drop=True), 
                          violations_details.reset_index(drop=True)], axis=1)
    
    return result_df

# Create a normalized violations dataframe
normalized_violations = parse_violations(df)

# Display the first few rows
normalized_violations.head()

,Inspection ID,violation_code,category,comment
0,1106427,32,"FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED",PROVIDE A DRAIN BOARD AT THREE COMPARTMENT SINK.
1,1106427,32,"FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED","OBSERVED RUSTY GREASE TRAP UNDER THE THREE COMPARTMENT SINK, INSTRUCTED TO PAINT AND MAINTAIN. ..."
2,1106427,34,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS...",INSTRUCTED TO SWEEP FLOOR INSIDE THE FURNACE AREA. OBSERVED AT SECOND FLOOR UNUSED CLASSROOMS W...
3,1106427,35,"WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-L...","OBSERVED DEAD BULB IN THE KITCHEN, INSTRUCTED TO REPLACE AND PROVIDE LIGHT SHIELD AT SAID FIXTURE"
4,2608378,1,"PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES",PIC DOESN'T HAVE A CERTIFIED FOOD MANAGERS CERTIFICATE MUST PROVIDE AND MAINTAIN.(PRIORITY FOUND...


In [ ]:
# Merge the original dataframe with the normalized violations dataframe if needed
df_merged = pd.merge(df, normalized_violations, on=['Inspection ID'], how='left') 
# Display the merged dataframe
df_merged.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,violation_code,category,comment
0,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789,Daycare Combo 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659,03/07/2013,License,Pass,"32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments:...",41.985390,-87.698734,"(41.98538950526786, -87.69873407149943)",32.0,"FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED",PROVIDE A DRAIN BOARD AT THREE COMPARTMENT SINK.
1,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789,Daycare Combo 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659,03/07/2013,License,Pass,"32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments:...",41.985390,-87.698734,"(41.98538950526786, -87.69873407149943)",32.0,"FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED","OBSERVED RUSTY GREASE TRAP UNDER THE THREE COMPARTMENT SINK, INSTRUCTED TO PAINT AND MAINTAIN. ..."
2,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789,Daycare Combo 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659,03/07/2013,License,Pass,"32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments:...",41.985390,-87.698734,"(41.98538950526786, -87.69873407149943)",34.0,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS...",INSTRUCTED TO SWEEP FLOOR INSIDE THE FURNACE AREA. OBSERVED AT SECOND FLOOR UNUSED CLASSROOMS W...
3,1106427,BLOOMING BUD DAYCARE,BLOOMING BUD DAYCARE,2215789,Daycare Combo 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,IL,60659,03/07/2013,License,Pass,"32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments:...",41.985390,-87.698734,"(41.98538950526786, -87.69873407149943)",35.0,"WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-L...","OBSERVED DEAD BULB IN THE KITCHEN, INSTRUCTED TO REPLACE AND PROVIDE LIGHT SHIELD AT SAID FIXTURE"
4,2608378,Babas Halal,Babas Halal,2684170,Restaurant,Risk 1 (High),7901 S DAMEN AVE,CHICAGO,IL,60620,12/03/2024,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES - Comments: PIC DOESN'T...",41.750189,-87.672986,"(41.750189342293375, -87.67298583977204)",1.0,"PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES",PIC DOESN'T HAVE A CERTIFIED FOOD MANAGERS CERTIFICATE MUST PROVIDE AND MAINTAIN.(PRIORITY FOUND...


In [36]:
from sqlite_utilities import *

In [ ]:
# Save the original dataframe to SQLite
df_to_sqlite(df, 'food_inspections.db', 'raw_inspections', if_exists='replace', index=False)

Successfully saved DataFrame to food_inspections.db as table 'raw_inspections'
Table contains 287237 rows and 17 columns


True

In [ ]:
query = 'drop table if exists  violation_categories'
query_to_df('food_inspections.db', query)

Error running query: 'NoneType' object is not iterable


In [ ]:
query = """
SELECT v.violation_code, vc.category, COUNT(*) as violation_count
FROM violations v
JOIN violation_categories vc ON v.violation_code = vc.violation_code
GROUP BY v.violation_code
ORDER BY violation_count DESC
LIMIT 10
"""
# Run the query and display the result
query_to_df('food_inspections.db', query)

,violation_code,category,violation_count
0,38,"INSECTS, RODENTS, & ANIMALS NOT PRESENT",144922
1,34,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS...",143396
2,33,"FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS",132974
3,35,APPROVED THAWING METHODS USED,131122
4,32,"FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED",110344
5,41,"PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING EQUIPMENT PROPERLY STORED",78476
6,55,"PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN",69851
7,36,"LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDED",68670
8,3,"MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING",53502
9,10,ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE,49280
